#Análise dos Dados de Conexão dos Usuários

##Análise do Levantamento Realizado


Fonte de dados: **Sistema Voalle**

Variáveis levantadas preliminarmente:

* Qtde de Incidentes Abertos Mês
* Tempo Médio de Atendimento
* Tipo de Incidentes
* Forma de Resolução
* Reincidência
* Tempo Médio entre Incidentes
* Atendimento SLA

**Análise das variáveis**<p>
Após a preparação e análise dos dados, optou-se por considerar apenas as variáveis: 
* **Qtde de Incidentes Abertos Ano**
* **Tempo Médio de Atendimento**
* **Tempo Médio entre Incidentes**

Como o estudo será com base nas informações do usuário, as informações que são específicas de um incidente como o caso do *tipo da solicitação* e da *forma de resolução* não deverão ser consideradas para este estudo de churn de clientes.

Não foi possível o cálculo das variáveis *Reincidência* e *Atendimento SLA* por que, atualmente, não existe uma padronização para registro dessas informações.

**Oportunidades Identificadas**<p>
Padronizar o registro das informações de reincidência de um determinado incidente.

Avaliar a relevância de estabelecer processo para cálculo do atendimento do SLA nos incidentes abertos pelos clientes.


##Importação de bibliotecas

In [ ]:
import pandas as pd

##Definição de Funções e Constantes

In [ ]:
#constantes
HORAS = 'HORAS'
MINUTOS = 'MINUTOS'
VALOR_MAXIMO = 9999999

In [ ]:
def calcula_tempo_atendimento(data_inicio, data_fim, unidade = 'HORAS'):

  if (pd.notnull(data_fim) == False):
    data_fim = pd.Timestamp.today(tz=data_inicio.tz)

  dias = pd.Timedelta(data_fim - data_inicio).days
  horas = pd.Timedelta(data_fim - data_inicio).seconds / (60 * 60)    # valor em horas
  minutos = pd.Timedelta(data_fim - data_inicio).seconds / 60         # minutos

  if unidade == HORAS:
    resultado = round((dias * 24) + horas) 
  else:
    resultado = round((dias * 24 * 60) + minutos)
  
  return resultado

In [ ]:
def calcula_tempo_medio_entre_incidentes(id_usuario):
  
  dados_aux = dados_calculo_tempo_entre_incidentes.loc[[id_usuario], :]
  dados_aux.sort_values('dt_hr_abertura', inplace=True)

  iteracao = 0
  contador = 0

  for idx, linha in dados_aux.iterrows():

    if iteracao == 0:
      data_encerramento_anterior = linha['dt_hr_conclusao']
    else:
      dias = pd.Timedelta(linha['dt_hr_abertura'] - data_encerramento_anterior).days
      contador = contador + dias
 
    iteracao += 1
    
  #calcula a média do tempo entre os incidentes do usuário
  
  #se houve apenas uma iteração, significa que não existem incidentes suficientes para o cálculo
  #então a função retorna o valor máximo predefino
  if iteracao == 1:
    resultado = VALOR_MAXIMO
  else:
    resultado = contador / (iteracao - 1)

    #se o tempo for 0, também é em razão de não ter incidentes suficientes para o cáculo
    if resultado == 0:
      resultado = VALOR_MAXIMO
    
    #print('Iteracao: {} - contador: {} - resultado: {}'.format(iteracao, contador, round(resultado)))
  
  return round(resultado)

##Carga dos Dados

In [ ]:
#dataset com dados dos incidentes
dados_origem = pd.read_csv('incidentes.csv')
dados_origem.head()

,assignment_id,contract_id,people_id,protocol,dt_hr_abertura,dt_hr_conclusao,status_solicitacao,tipo_solicitacao,contexto,problema
0,297492,11844,13670,236908,2021-10-25 14:45:32.000,2021-10-25 14:46:35.000,Encerramento,Solicitação de Desbloqueio,NaN,NaN
1,297491,22828,12388,236907,2021-10-25 14:45:23.000,2021-10-25 14:46:53.000,Encerramento,Dúvida ou Informação,NaN,NaN
2,297490,21269,31400,236906,2021-10-25 14:44:46.000,2021-10-25 14:47:19.000,Encerramento,Solicitação de Desbloqueio,NaN,NaN
3,297486,22092,32793,236902,2021-10-25 14:32:34.000,2021-10-25 14:36:41.000,Encerramento,Solicitação de Desbloqueio,NaN,NaN
4,297487,8802,9171,236903,2021-10-25 14:32:00.000,2021-10-25 14:41:39.000,Encerramento,Dúvida ou Informação,NaN,NaN


In [ ]:
linhas, colunas = dados_origem.shape
print('A fonte de dados de incidentes possui %d linhas e %d colunas.' % (linhas, colunas))

A fonte de dados de incidentes possui 126734 linhas e 10 colunas.


##Tratamento dos Dados


In [ ]:
#dataset principal da análise com as informações dos incidentes do usuário
dados = dados_origem.copy()
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126734 entries, 0 to 126733
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   assignment_id       126734 non-null  int64 
 1   contract_id         126734 non-null  int64 
 2   people_id           126734 non-null  int64 
 3   protocol            126734 non-null  int64 
 4   dt_hr_abertura      126734 non-null  object
 5   dt_hr_conclusao     126732 non-null  object
 6   status_solicitacao  126734 non-null  object
 7   tipo_solicitacao    126734 non-null  object
 8   contexto            50637 non-null   object
 9   problema            50440 non-null   object
dtypes: int64(4), object(6)
memory usage: 9.7+ MB


In [ ]:
#Conversão das colunas de data/hora
dados['dt_hr_abertura'] = pd.to_datetime(dados['dt_hr_abertura'], errors='coerce')
dados['dt_hr_conclusao'] = pd.to_datetime(dados['dt_hr_conclusao'], errors='coerce')

#Criação de nova coluna
dados['ano'] = dados['dt_hr_abertura'].dt.strftime("%Y")
dados['mes'] = dados['dt_hr_abertura'].dt.strftime("%m")
dados['tempo_total'] = dados.apply(lambda x: calcula_tempo_atendimento(x['dt_hr_abertura'], x['dt_hr_conclusao'], HORAS), axis=1) 

#exclui colunas não utilizadas
dados.drop(['status_solicitacao', 'contexto', 'problema', 'tipo_solicitacao'], axis=1, inplace=True)

dados.head()

,assignment_id,contract_id,people_id,protocol,dt_hr_abertura,dt_hr_conclusao,ano,mes,tempo_total
0,297492,11844,13670,236908,2021-10-25 14:45:32,2021-10-25 14:46:35,2021,10,0
1,297491,22828,12388,236907,2021-10-25 14:45:23,2021-10-25 14:46:53,2021,10,0
2,297490,21269,31400,236906,2021-10-25 14:44:46,2021-10-25 14:47:19,2021,10,0
3,297486,22092,32793,236902,2021-10-25 14:32:34,2021-10-25 14:36:41,2021,10,0
4,297487,8802,9171,236903,2021-10-25 14:32:00,2021-10-25 14:41:39,2021,10,0


In [ ]:
#prepara o dataset utilizado para o cálculo do tempo entre incidentes
dados_calculo_tempo_entre_incidentes = dados.copy();

dados_calculo_tempo_entre_incidentes = dados_calculo_tempo_entre_incidentes[['people_id', 'dt_hr_abertura', 'dt_hr_conclusao']]

dados_calculo_tempo_entre_incidentes.set_index('people_id', inplace=True)
dados_calculo_tempo_entre_incidentes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126734 entries, 13670 to 385
Data columns (total 2 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   dt_hr_abertura   126734 non-null  datetime64[ns]
 1   dt_hr_conclusao  126732 non-null  datetime64[ns]
dtypes: datetime64[ns](2)
memory usage: 2.9 MB


In [ ]:
#prepara o dataset que irá agrupar o resultado final por usuário
dados_usuario = pd.DataFrame(dados['people_id'].unique())
dados_usuario.columns = ['people_id']
dados_usuario.set_index('people_id', inplace=True)

print(f'Número de {dados_usuario.shape[0]} usuários no total.')

Número de 13296 usuários no total.


##Cálculo das Variáveis

In [ ]:
#avalia os valores existentes no dataset para determinar a relevância ou não das informações para o estudo
dados_origem['tipo_solicitacao'].unique().shape[0]

116

Como o estudo será com base nas informações do usuário, as informações que são específicas de um incidente como o caso do **tipo da solicitação** e da **forma de resolução** não deverão ser consideradas para este estudo de churn de clientes.

###Tempo Médio para Encerramento do Incidente

In [ ]:
#Agrupa as informações dos incidentes por usuário e calcula a média do tempo total do atendimento
dados_usuario_tempo_medio = dados.groupby(['people_id'])['tempo_total'].agg(['mean'])

dados_usuario_tempo_medio.columns = ['horas_resolucao_incidente']

dados_usuario_tempo_medio.head()

,horas_resolucao_incidente
people_id,
1,385.000000
12,3.833333
13,734.000000
14,275.833333
15,382.166667


###Média de Incidentes por ano

In [ ]:
#Agrupa as informações dos incidentes por usuário e ano para proceder com a contagem anual
dados_usuario_incidentes_ano = dados.groupby(['people_id', 'ano'])['assignment_id'].agg(['count'])
dados_usuario_incidentes_ano.columns = ['incidentes_ano']
dados_usuario_incidentes_ano.reset_index(inplace=True)

dados_usuario_incidentes_ano.head()

,people_id,ano,incidentes_ano
0,1,2018,1
1,12,2018,6
2,13,2018,3
3,13,2019,1
4,14,2018,6


###Tempo Médio Entre Incidentes

In [ ]:
dados_tempo_entre_incidentes = dados_usuario.copy().reset_index()

dados_tempo_entre_incidentes['dias_entre_incidentes'] = dados_tempo_entre_incidentes.apply(
    lambda x: calcula_tempo_medio_entre_incidentes(x['people_id']), axis=1)

dados_tempo_entre_incidentes.set_index('people_id', inplace=True)

dados_tempo_entre_incidentes.head()

,dias_entre_incidentes
people_id,
13670,490
12388,42
31400,202
32793,136
9171,505


##Preparação do Dataset Final

In [ ]:
#Agrega as informações do tempo médio para encerramento dos incidentes
dados_usuario = dados_usuario.join(dados_usuario_tempo_medio)

#Agrega as informações do tempo médio entre incidentes
dados_usuario = dados_usuario.join(dados_tempo_entre_incidentes)

#Agrupa por usuário para calcular a média anual de incidentes abertos
dados_usuario_incidentes_ano_media = pd.DataFrame(dados_usuario_incidentes_ano.groupby(['people_id'])['incidentes_ano'].agg('mean').round(3))
dados_usuario_incidentes_ano_media.columns = ['incidentes_ano']

dados_usuario = dados_usuario.join(dados_usuario_incidentes_ano_media)

dados_usuario.head()

,horas_resolucao_incidente,dias_entre_incidentes,incidentes_ano
people_id,,,
13670,42.000000,490,5.667
12388,54.636364,42,11.000
31400,23.666667,202,6.000
32793,0.000000,136,6.000
9171,275.312500,505,4.000


In [ ]:
dados_usuario.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13296 entries, 13670 to 385
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   horas_resolucao_incidente  13296 non-null  float64
 1   dias_entre_incidentes      13296 non-null  int64  
 2   incidentes_ano             13296 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 1.0 MB


In [ ]:
dados_usuario.to_csv('dados_usuario_incidentes.csv', sep=';', decimal=',')